In [1]:
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer
import re
from transformers import AdamWeightDecay
from datasets import Dataset
import math
import numpy as np
from transformers import DataCollatorForSeq2Seq
from transformers import TFAutoModelForSeq2SeqLM
import torch
import matplotlib.pyplot as plt
import evaluate
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration
import torch

2023-11-22 17:16:51.652690: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-22 17:16:51.769213: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-22 17:16:51.769870: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-22 17:16:57.827044: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def json_to_dataframe(filename):
    f = open(filename)
    df = pd.read_json(f)
    print(df.iloc[0])
    f.close()
    df["masked headline"]= df.apply(lambda x: x['masked headline'].replace('____', str(x['ans'])), axis=1)
    print(df.iloc[0])
    df['news'] = df['news'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))
    #df['masked headline'] = df['masked headline'].str.replace('____', '<extra_id_0> ') + ' </s>'
    df['text'] = df[['news', 'masked headline']].apply(" ".join, axis=1)
    df = df.astype(str)
    return df

In [3]:
train_data = json_to_dataframe('Train_Numerical_Reasoning.json')

news               (Oct 7, 2014  12:40 PM CDT) As of Jan. 1, Walm...
masked headline    ____K Walmart Part-Timers to Lose Health Insur...
calculation                                     Paraphrase(30,000,K)
ans                                                               30
Name: 0, dtype: object
news               (Oct 7, 2014  12:40 PM CDT) As of Jan. 1, Walm...
masked headline     30K Walmart Part-Timers to Lose Health Insurance
calculation                                     Paraphrase(30,000,K)
ans                                                               30
Name: 0, dtype: object


In [4]:
train_data.head()

news  \
0   As of Jan. 1, Walmart will no longer offer 30...   
1   Dax Shepard and Kristen Bell got married at t...   
2   Nancy Reagan, the helpmate, backstage adviser...   
3   American Airlines faces FAA fines of more tha...   
4   Ingrid Lyne, the Seattle mom allegedly murder...   

                                     masked headline            calculation  \
0   30K Walmart Part-Timers to Lose Health Insurance   Paraphrase(30,000,K)   
1     Dax Shepard: Wedding to Kristen Bell Cost $142              Copy(142)   
2                            Nancy Reagan Dead at 94               Copy(94)   
3  American Airlines Faces $7M Fine for Safety Vi...                Copy(7)   
4   $222K Raised for Kids of Mom Dismembered on Date  Paraphrase(222,000,K)   

   ans                                               text  
0   30   As of Jan. 1, Walmart will no longer offer 30...  
1  142   Dax Shepard and Kristen Bell got married at t...  
2   94   Nancy Reagan, the helpmate, backstage adviser...  
3    7   American Airlines faces FAA fines of more tha...  
4  222   Ingrid Lyne, the Seattle mom allegedly murder...

In [5]:
dataset = Dataset.from_pandas(train_data)
dataset = dataset.train_test_split(test_size=0.2)

In [6]:
#model_name = "ybagoury/flan-t5-base-tldr_news"
#model_name = "Michau/t5-base-en-generate-headline"
model_name = 'MBZUAI/LaMini-Flan-T5-783M'

In [2]:
model_name = 'MBZUAI/LaMini-Flan-T5-783M'
tokenizer = AutoTokenizer.from_pretrained(model_name)
optimizer = AdamWeightDecay(learning_rate=5e-5, weight_decay_rate=0.01)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [10]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                           model=model_name,
                                           return_tensors="tf")
#optimizer = AdamWeightDecay(learning_rate=1e-4, weight_decay_rate=0.01)
#optimizer = AdamWeightDecay(learning_rate=7e-5, weight_decay_rate=0.01)
optimizer = AdamWeightDecay(learning_rate=5e-5, weight_decay_rate=0.01)

In [11]:
def preprocess_function2(examples):   
    inputs = tokenizer(examples['calculation'],max_length=248,  truncation=True)
    #labels = tokenizer(text_target=examples["headline"], max_length=128, truncation=True)
    labels = tokenizer(text_target=examples["ans"], max_length=20, truncation=True)
    inputs["labels"] = labels["input_ids"]
    return inputs

In [12]:
tokenized2 = dataset.map(preprocess_function2, batched=True, #num_proc=4,
                        remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/16925 [00:00<?, ? examples/s]

Map:   0%|          | 0/4232 [00:00<?, ? examples/s]

In [14]:
tf_train_set2 = model2.prepare_tf_dataset(
    tokenized2["train"],
    shuffle=True,
    batch_size=3,
    collate_fn=data_collator,
)

tf_test_set2 = model2.prepare_tf_dataset(
    tokenized2["test"],
    shuffle=False,
    batch_size=3,
    collate_fn=data_collator,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [13]:
model2 = TFAutoModelForSeq2SeqLM.from_pretrained(model_name, from_pt=True)

2023-11-18 13:42:23.252142: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15520 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:0b:00.0, compute capability: 6.0
/home/u3/hinokicrum/.local/lib/python3.9/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. in

In [15]:
model2.compile(optimizer=optimizer)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
history2 = model2.fit(x=tf_train_set2, validation_data=tf_test_set2, callbacks=[callback], epochs=8)

Epoch 1/8
5641/5641 [==============================] - 1471s 248ms/step - loss: 0.1828 - val_loss: 0.1106
Epoch 2/8
5641/5641 [==============================] - 1380s 245ms/step - loss: 0.1145 - val_loss: 0.1065
Epoch 3/8
5641/5641 [==============================] - 1376s 244ms/step - loss: 0.0902 - val_loss: 0.1037
Epoch 4/8
5641/5641 [==============================] - 1377s 244ms/step - loss: 0.0789 - val_loss: 0.1099


In [16]:
model2.save('Lamini182.keras')


/home/u3/hinokicrum/.local/lib/python3.9/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [8]:
model2 = tf.keras.models.load_model('Lamini182.keras')

2023-11-22 17:17:39.396256: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/u3/hinokicrum/.local/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


/home/u3/hinokicrum/.local/lib/python3.8/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [6]:
f = open('Dev_Numerical_Reasoning.json')
df = pd.read_json(f)
df['news'] = df['news'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))
df['masked headline'] = df['masked headline'].str.replace('____', '<extra_id_0> ') + ' </s>'
df['text'] = df[['news', 'masked headline']].apply(" ".join, axis=1)
df = df.astype(str)
#df = df.iloc[:200]
#df = df.iloc[100:500]
f.close()
test_data = Dataset.from_pandas(df)

In [3]:
cal = pd.read_excel('Lamini2.xlsx')

In [4]:
def t5_predictions2(data, tokenizer, model):


    inputs = tokenizer.encode_plus(data, add_special_tokens=True,
                                   return_tensors="pt").input_ids
    # input_ids = inputs['input_ids'].to(device)
    # attention_masks = inputs['attention_mask']

    outputs = model.generate(inputs,
                             max_length=128, num_beams=5, early_stopping=True)


    res = ''
    results = tokenizer.decode(outputs[0][1:], skip_special_tokens=False,
                               clean_up_tokenization_spaces=False)
    res = re.findall(r"([\d:,./]+)", results)
    return res[0]

In [9]:
false=0
predictions = []
for x,y in zip(cal['calc'], test_data['ans']):
    ans = t5_predictions2(x, tokenizer, model2)
    predictions.append(ans)
    if ans != y:
        print(ans)
        print(y)
        print("")
        false += 1

2023-11-22 17:52:00.669594: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x233ae740 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-11-22 17:52:00.669692: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-11-22 17:52:00.717562: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-22 17:52:00.743931: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5
9

1.1
2

2010
2000

300
285

2
5

2
59

28
2.8

4
4.5

700
350

37
31

8
47

79.5
160

3
4

121
1892

2
17

28
2

96,824
12

4
26

91
62

3.5
3

10
15.5

109
100

4
2

25
27

200
800

2
10

26.6
28.6

11.5
11

11
2

633
643

3.1
3

229
230

1.27
127

3
20

3
1

400
1,000

513
500

2012
2010

6
51

2
3

2013
1981

1,068
1,000

169,000
19

9.5
9

50
40

12
10

10
12

6
25

40
30

10
2

33
30

18
21

70
72

520
500

3
2

2
31

20
12

98
9.8

28
73

27
27.2

13
15

23,334
23,344

2
30

23.3
23

2.7
2.77

1950
1958

2
4.4

40
560

6
5

45
50

2008
1964

8.5
5

400
10

2016
2014

258
248

1
2

8.81
89

306
20

2004
9/11

3
1

10
4

204.6
205

1.5
1

50
99

1,08
1,018

4
5

25.5
25

47
150

7.7
8

10
25

10.9
11

187
2

6.2
6

1.11
1.1

43.7
45

54
24

23.3
22.3

1
2

40
43

13
2

30
8

85
30

630
730

108
11

10.5
10.10

5
88

2
1.9

3
4

40.7
48.7

3
4

2011
2000

3
2

300
299

480
500

2
78

5
6

5
11

99/100
0.99

40
39

5
2

2014
2

7
787

800
1200

273
270

10
06

100
459

3.12
3

20

In [10]:
false

283

In [11]:
print(1-false/len(test_data))

0.8899688958009331


In [12]:
with open('predictions.txt', 'w+') as f:
     
    for prediction in predictions:
        f.write('%s\n' %prediction)
     
f.close()

In [13]:
from sklearn.metrics import accuracy_score
accuracy_score(df['ans'], predictions)

0.8899688958009331